In [ ]:
import pandas as pd 
from google.cloud import storage 
from google.cloud import bigquery
import mintapi as mintapi

def get_mint_transactions():
    
    mint_auth = mintapi.Mint(
    'email',
    'password',
    mfa_method = 'sms',
    mfa_input_callback=None,
    mfa_token=None,
    intuit_account=None,
    headless=False,
    session_path=None,
    imap_account=None,
    imap_password=None,
    imap_server=None,
    imap_folder='INBOX',
    wait_for_sync=False, 
    wait_for_sync_timeout=300,
    use_chromedriver_on_path=False
    )
    
    mint = mint_auth.get_transactions(include_investment=True)
    mint_auth.close()
    
    mint_df = pd.DataFrame(mint)
    mint_df = mint_df.drop('labels', axis=1) 
    mint_df = mint_df.drop('notes', axis=1)
    mint_df['description'] = mint_df['description'].str.title()
    mint_df['original_description'] = mint_df['original_description'].str.title()
    mint_df['transaction_type'] = mint_df['transaction_type'].str.title()
    mint_df['category'] = mint_df['category'].str.title()
    mint_df['paycheck_amount_remaining_week'] = round( - mint_df['amount'],2)
    mint_df['paycheck_amount_remaining_month'] = round(**** - mint_df['amount'],2)
    mint_df['discretionary_remaining_month'] = round(**** - mint_df['amount'],2)
    mint_df['discretionary_remaining_year'] = round(***** - mint_df['amount'],2)
    
    client = bigquery.Client()
    dataset_id = 'mint'
    table_id = 'transactions'

    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition='WRITE_TRUNCATE'
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.autodetect=True
    job_config.ignore_unknown_values=True 

    job = client.load_table_from_dataframe(
    mint_df,
    table_ref,
    location='US',
    job_config=job_config)
    
    job.result()
    return 'Transactions table loaded.'
    
get_mint_transactions()